This notebook creates a VM in the user's project with the airflow scheduler and webserver. A default GCP zone for the VM has been chosen (below). Feel free to change this as desired.

In [1]:
# Get the latest datalab version. Restart the kernel.
!pip install --upgrade --force-reinstall datalab

  Using cached pytz-2017.3-py2.py3-none-any.whl
  Using cached ipykernel-4.5.2-py2.py3-none-any.whl
  Using cached scikit_learn-0.18.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached pandas-0.22.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached pandas_profiling-1.4.1-py2.py3-none-any.whl
  Using cached requests-2.9.1-py2.py3-none-any.whl
  Using cached jsonschema-2.6.0-py2.py3-none-any.whl
  Using cached scikit_image-0.13.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached urllib3-1.22-py2.py3-none-any.whl
  Using cached futures-3.0.5-py2-none-any.whl
  Using cached google_auth_httplib2-0.0.2-py2.py3-none-any.whl
  Using cached mock-2.0.0-py2.py3-none-any.whl
  Using cached six-1.10.0-py2.py3-none-any.whl
  Using cached python_dateutil-2.5.0-py2.py3-none-any.whl
  Using cached google_api_python_client-1.6.2-py2.py3-none-any.whl
  Using cached google_cloud-0.30.0-py2.py3-none-any.whl
  Using cached numpy-1.14.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached scipy-1.0.0-cp27-cp27mu-ma

  Using cached Pygments-2.2.0-py2.py3-none-any.whl
  Using cached pexpect-4.3.1-py2.py3-none-any.whl
  Using cached pathlib2-2.3.0-py2.py3-none-any.whl
  Using cached setuptools-38.4.0-py2.py3-none-any.whl
  Using cached prompt_toolkit-1.0.15-py2-none-any.whl
  Using cached enum34-1.1.6-py2-none-any.whl
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl
  Using cached singledispatch-3.4.0.3-py2.py3-none-any.whl
  Using cached certifi-2018.1.18-py2.py3-none-any.whl
  Using cached backports_abc-0.5-py2.py3-none-any.whl
  Using cached pyzmq-16.0.4-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached jupyter_core-4.4.0-py2.py3-none-any.whl
  Using cached cachetools-2.0.1-py2.py3-none-any.whl
  Using cached google_gax-0.15.16-py2.py3-none-any.whl
  Using cached google_resumable_media-0.3.1-py2.py3-none-any.whl
  Using cached protobuf-3.5.1-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached grpcio-1.9.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached ptyprocess-0.5.2-py2.py3-none-any.whl


  Found existing installation: traitlets 4.3.2
    Uninstalling traitlets-4.3.2:
      Successfully uninstalled traitlets-4.3.2
  Found existing installation: backports.shutil-get-terminal-size 1.0.0
    Uninstalling backports.shutil-get-terminal-size-1.0.0:
      Successfully uninstalled backports.shutil-get-terminal-size-1.0.0
  Found existing installation: Pygments 2.2.0
    Uninstalling Pygments-2.2.0:
      Successfully uninstalled Pygments-2.2.0
  Found existing installation: ptyprocess 0.5.2
    Uninstalling ptyprocess-0.5.2:
      Successfully uninstalled ptyprocess-0.5.2
  Found existing installation: pexpect 4.3.1
    Uninstalling pexpect-4.3.1:
      Successfully uninstalled pexpect-4.3.1
  Found existing installation: setuptools 38.4.0
    Uninstalling setuptools-38.4.0:
      Successfully uninstalled setuptools-38.4.0
  Found existing installation: wcwidth 0.1.7
    Uninstalling wcwidth-0.1.7:
      Successfully uninstalled wcwidth-0.1.7
  Found existing installation: prom

      Successfully uninstalled google-cloud-datastore-1.4.0
  Found existing installation: proto-google-cloud-error-reporting-v1beta1 0.15.3
    Uninstalling proto-google-cloud-error-reporting-v1beta1-0.15.3:
      Successfully uninstalled proto-google-cloud-error-reporting-v1beta1-0.15.3
  Found existing installation: gapic-google-cloud-error-reporting-v1beta1 0.15.3
    Uninstalling gapic-google-cloud-error-reporting-v1beta1-0.15.3:
      Successfully uninstalled gapic-google-cloud-error-reporting-v1beta1-0.15.3
  Found existing installation: proto-google-cloud-logging-v2 0.91.3
    Uninstalling proto-google-cloud-logging-v2-0.91.3:
      Successfully uninstalled proto-google-cloud-logging-v2-0.91.3
  Found existing installation: gapic-google-cloud-logging-v2 0.91.3
    Uninstalling gapic-google-cloud-logging-v2-0.91.3:
      Successfully uninstalled gapic-google-cloud-logging-v2-0.91.3
  Found existing installation: google-cloud-logging 1.4.0
    Uninstalling google-cloud-logging-1.

In [2]:
zone='us-central1-b'

In [3]:
from google.datalab import Context
import google.datalab.storage as storage

project = Context.default().project_id
vm_name = 'datalab-airflow'

# The name of this GCS bucket follows a convention between this notebook and 
# the 'BigQuery Pipeline' tutorial notebook, so don't change this.
gcs_dag_bucket_name = project + '-' + vm_name
gcs_dag_bucket = storage.Bucket(gcs_dag_bucket_name)
gcs_dag_bucket.create()

Google Cloud Storage Bucket gs://<project-id>-datalab-airflow

In [4]:
vm_startup_script_contents = """#!/bin/bash
apt-get update
apt-get --assume-yes install python-pip

pip install datalab==1.1.2
pip install apache-airflow==1.9.0
pip install pandas-gbq==0.3.0

export AIRFLOW_HOME=/airflow
export AIRFLOW__CORE__DAGS_ARE_PAUSED_AT_CREATION=False
export AIRFLOW__CORE__LOAD_EXAMPLES=False
airflow initdb
airflow scheduler &
airflow webserver -p 8080 &d

# We append a gsutil rsync command to the cron file and have this run every minute to sync dags.
PROJECT_ID=$(gcloud info --format="get(config.project)")
GCS_DAG_BUCKET=$PROJECT_ID-datalab-airflow
AIRFLOW_CRON=temp_crontab.txt
crontab -l > $AIRFLOW_CRON
DAG_FOLDER="dags"
LOCAL_DAG_PATH=$AIRFLOW_HOME/$DAG_FOLDER
mkdir $LOCAL_DAG_PATH
echo "* * * * * gsutil rsync gs://$GCS_DAG_BUCKET/$DAG_FOLDER $LOCAL_DAG_PATH" >> $AIRFLOW_CRON
crontab $AIRFLOW_CRON
rm $AIRFLOW_CRON
EOF
"""
vm_startup_script_file_name = 'vm_startup_script.sh'
script_file = open(vm_startup_script_file_name, 'w')
script_file.write(vm_startup_script_contents)
script_file.close()
import subprocess
print subprocess.check_output([
    'gcloud', 'compute', '--project', project, 'instances', 'create', vm_name, 
    '--zone', zone,
    '--machine-type', 'n1-standard-1',
    '--network', 'default',
    '--maintenance-policy', 'MIGRATE',
    '--scopes', 'https://www.googleapis.com/auth/cloud-platform',
    '--image', 'debian-9-stretch-v20171025',
    '--min-cpu-platform', 'Automatic',
    '--image-project', 'debian-cloud',
    '--boot-disk-size', '10',
    '--boot-disk-type', 'pd-standard',
    '--boot-disk-device-name', vm_name,
    '--metadata-from-file', 'startup-script=' + vm_startup_script_file_name])

NAME               ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
datalab-airflow  us-central1-b  n1-standard-1               10.240.0.5   35.188.88.29  RUNNING



# Cleanup


In [5]:
# The following cleans up the VM and associated GCS bucket. Uncomment and run.
#!gsutil rm -r gs://$gcs_dag_bucket_name
#!gcloud compute instances delete datalab-airflow --zone us-central1-b --quiet

# This just verifies that cleanup actually worked. Uncomment and run. Should 
# show an error like "BucketNotFoundException: 404 ...". 
#!gsutil ls gs://$gcs_dag_bucket_name